In [28]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Setup

In [29]:
import bigframes.pandas as bpd

In [30]:
df = bpd.read_gbq("bigquery-public-data.baseball.schedules")[["homeTeamName", "awayTeamName", "duration_minutes"]]
df.peek()

/usr/local/google/home/shobs/code/bigframes/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3550: UserWarning: Reading cached table from 2024-06-28 02:49:31.716256+00:00 to avoid incompatibilies with previous reads of this table. To read the latest version, set `use_cache=False` or close the current session with Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)


,homeTeamName,awayTeamName,duration_minutes
50,Rays,Rangers,181
72,Phillies,Pirates,192
89,Mariners,Blue Jays,183
351,Astros,Angels,212
382,Royals,Yankees,259


# Notes

* The API reference documentation for the `remote_function` can be found at
  https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.session.Session#bigframes_session_Session_remote_function

* More code samples for `remote_function` can be found in the BigQuery
  DataFrames API reference documentation, e.g.
  * https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.series.Series#bigframes_series_Series_apply
  * https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.dataframe.DataFrame#bigframes_dataframe_DataFrame_map
  * https://cloud.google.com/python/docs/reference/bigframes/latest/bigframes.dataframe.DataFrame#bigframes_dataframe_DataFrame_apply

* The following examples are only for the purpose of demonstrating
`remote_function` usage. They are not necessarily the best way to achieve the
end result.

* In the examples in this notebook we are using `reuse=False` just as a caution
  to avoid concurrent runs of this notebook in the same google cloud project
  stepping over each other's remote function deployment. It may not be neccesary
  in a simple use case.

# Self-contained function

Let's consider a scenario where we want to categorize the matches as short,
medium or long duration based on the `duration_minutes` column.

In [31]:
@bpd.remote_function(reuse=False)
def duration_category(duration_minutes: int) -> str:
    if duration_minutes < 90:
        return "short"
    elif duration_minutes < 180:
        return "medium"
    else:
        return "long"

print(f"Created cloud function '{duration_category.bigframes_cloud_function}' and BQ remote function '{duration_category.bigframes_remote_function}'.")

Created cloud function 'projects/bigframes-dev/locations/us-central1/functions/bigframes-862150459da5240a6df1ce01c59b32d8-em4ibov0' and BQ remote function 'bigframes-dev._1b6c31ff1bcd5d2f6d86833cf8268317f1b12d57.bigframes_862150459da5240a6df1ce01c59b32d8_em4ibov0'.


In [32]:
df1 = df.assign(duration_cat=df["duration_minutes"].apply(duration_category))
df1.peek()

,homeTeamName,awayTeamName,duration_minutes,duration_cat
1911,Dodgers,Angels,132,medium
2365,Athletics,Angels,134,medium
1977,Athletics,Angels,139,medium
554,Cubs,Angels,142,medium
654,Astros,Angels,143,medium


# Function referring to variables outside the function body

Let's consider a slight variation of the earlier example where the labels for
the short, medium and long duration matches are defined outside the function
body. They would be captured at the time of `remote_function` deployment and
any change in their values in the notebook after the deployment will not
automatically propagate to the `remote_function`.

In [33]:
DURATION_CATEGORY_SHORT = "S"
DURATION_CATEGORY_MEDIUM = "M"
DURATION_CATEGORY_LONG = "L"

In [34]:
@bpd.remote_function(reuse=False)
def duration_category(duration_minutes: int) -> str:
    if duration_minutes < 90:
        return DURATION_CATEGORY_SHORT
    elif duration_minutes < 180:
        return DURATION_CATEGORY_MEDIUM
    else:
        return DURATION_CATEGORY_LONG

print(f"Created cloud function '{duration_category.bigframes_cloud_function}' and BQ remote function '{duration_category.bigframes_remote_function}'.")

Created cloud function 'projects/bigframes-dev/locations/us-central1/functions/bigframes-f3231b74ec807496f4894218d5d40ed5-688mx7hi' and BQ remote function 'bigframes-dev._1b6c31ff1bcd5d2f6d86833cf8268317f1b12d57.bigframes_f3231b74ec807496f4894218d5d40ed5_688mx7hi'.


In [35]:
df1 = df.assign(duration_cat=df["duration_minutes"].apply(duration_category))
df1.peek()

,homeTeamName,awayTeamName,duration_minutes,duration_cat
1911,Dodgers,Angels,132,M
2365,Athletics,Angels,134,M
1977,Athletics,Angels,139,M
554,Cubs,Angels,142,M
654,Astros,Angels,143,M


# Function referring to imports (built-in) outside the function body

Let's consider a scenario in which we want to categorize the matches in terms of
hour buckets. E.g. a match finishing in 0-60 minutes would be in 1h category,
61-120 minutes in 2h category and so on. The function itself makes use of the
`math` module (a built-in module in a standard python installation) which
happens to be imported outside the function body, let's say in one of the
previous cells.

Later in the notebook we will see another example with a third-party module.

In [36]:
import math

In [37]:
@bpd.remote_function(reuse=False)
def duration_category(duration_minutes: int) -> str:
    duration_hours = math.ceil(duration_minutes / 60)
    return f"{duration_hours}h"

print(f"Created cloud function '{duration_category.bigframes_cloud_function}' and BQ remote function '{duration_category.bigframes_remote_function}'.")

Created cloud function 'projects/bigframes-dev/locations/us-central1/functions/bigframes-9b20b0257558a42da610d8998022c25e-7k62x9l6' and BQ remote function 'bigframes-dev._1b6c31ff1bcd5d2f6d86833cf8268317f1b12d57.bigframes_9b20b0257558a42da610d8998022c25e_7k62x9l6'.


In [38]:
df1 = df.assign(duration_cat=df["duration_minutes"].apply(duration_category))
df1.peek()

,homeTeamName,awayTeamName,duration_minutes,duration_cat
1911,Dodgers,Angels,132,3h
2365,Athletics,Angels,134,3h
1977,Athletics,Angels,139,3h
554,Cubs,Angels,142,3h
654,Astros,Angels,143,3h


# Function referring to another function outside the function body

In this example let's create a `remote_function` from a function
`duration_category` which depends upon another function `get_hour_ceiling`,
which further depends on another function `get_minutes_in_hour`. This dependency
chain could be even longer in a real world example. The behaviors of the
dependencies would be captured at the time of the remote function
deployment.

Please ntoe that any changes in those functions in the notebook after the
deployment would not automatically propagate to the remote function.

In [39]:
import math

def get_minutes_in_hour():
    return 60

def get_hour_ceiling(minutes):
    return math.ceil(minutes / get_minutes_in_hour())

In [40]:
@bpd.remote_function(reuse=False)
def duration_category(duration_minutes: int) -> str:
    duration_hours = get_hour_ceiling(duration_minutes)
    return f"{duration_hours} hrs"

print(f"Created cloud function '{duration_category.bigframes_cloud_function}' and BQ remote function '{duration_category.bigframes_remote_function}'.")

Created cloud function 'projects/bigframes-dev/locations/us-central1/functions/bigframes-b54aa0aa752af6a3bd6d9d529dac373b-h4lgpy4y' and BQ remote function 'bigframes-dev._1b6c31ff1bcd5d2f6d86833cf8268317f1b12d57.bigframes_b54aa0aa752af6a3bd6d9d529dac373b_h4lgpy4y'.


In [41]:
df1 = df.assign(duration_cat=df["duration_minutes"].apply(duration_category))
df1.peek()

,homeTeamName,awayTeamName,duration_minutes,duration_cat
1911,Dodgers,Angels,132,3 hrs
2365,Athletics,Angels,134,3 hrs
1977,Athletics,Angels,139,3 hrs
554,Cubs,Angels,142,3 hrs
654,Astros,Angels,143,3 hrs


# Function requiring external packages

In this example let's say we want to redact the `homeTeamName` values, and we
choose to use a third party library `cryptography`. Any third party dependencies
can be specified in [pip format](https://pip.pypa.io/en/stable/reference/requirements-file-format/)
(with or without version number) as a list via the `packages` parameter.

In [42]:
@bpd.remote_function(reuse=False, packages=["cryptography"])
def get_hash(input: str) -> str:
    from cryptography.fernet import Fernet

    # handle missing value
    if input is None:
        input = ""

    key = Fernet.generate_key()
    f = Fernet(key)
    return f.encrypt(input.encode()).decode()

In [43]:
df1 = df.assign(homeTeamNameRedacted=df["homeTeamName"].apply(get_hash))
df1.peek()

,homeTeamName,awayTeamName,duration_minutes,homeTeamNameRedacted
719,Astros,Angels,180,gAAAAABmflbKCFygsmoTzFkUCObFSBJG29Ksk8HEtk82ib...
2295,Astros,Angels,204,gAAAAABmflbKv-XzIxcNS92RO4fXYIAwA0kGWsAy-tI5fm...
1126,Astros,Angels,176,gAAAAABmflbJdjgpqnfvmklU7Zg3NJUqlTMYMs44dLEkwg...
294,Astros,Angels,189,gAAAAABmflbKmfBh4P3FnwyiIpVFek9TzF4GzwP_5rQmkv...
351,Astros,Angels,212,gAAAAABmflbJ_mzqao9i7BtoYlMpb6y3bV3x7-cYuWGxsT...


# Function referring to imports (third-party) outside the function body

In this scenario the function depends on a third party library and the module
from the third party library used in the function is imported outside the
function body in a previous cell. Below is such an example where the third-party
dependency is `humanize` and its module of the same name is imported outside the
function body.

In [44]:
import datetime as dt
import humanize

In [45]:
@bpd.remote_function(reuse=False, packages=["humanize"])
def duration_category(duration_minutes: int) -> str:
    timedelta = dt.timedelta(minutes=duration_minutes)
    return humanize.naturaldelta(timedelta)

print(f"Created cloud function '{duration_category.bigframes_cloud_function}' and BQ remote function '{duration_category.bigframes_remote_function}'.")

Created cloud function 'projects/bigframes-dev/locations/us-central1/functions/bigframes-0879f72acd9b8ede460b69c5a8cc0dcb-edxlst27' and BQ remote function 'bigframes-dev._1b6c31ff1bcd5d2f6d86833cf8268317f1b12d57.bigframes_0879f72acd9b8ede460b69c5a8cc0dcb_edxlst27'.


In [46]:
df1 = df.assign(duration_cat=df["duration_minutes"].apply(duration_category))
df1.peek()

,homeTeamName,awayTeamName,duration_minutes,duration_cat
1911,Dodgers,Angels,132,2 hours
2365,Athletics,Angels,134,2 hours
1977,Athletics,Angels,139,2 hours
554,Cubs,Angels,142,2 hours
654,Astros,Angels,143,2 hours
